In this notebook, we attempt to fit two logistic regression models, one with surface-specific intercepts, and the other without them.  We fit both models through SKLearn.  We fit "set models" only.  We discover that both models work reasonably well, with the set data providing a small lift in performance in the part of the validation set that has surface information.

We also implement different regularization strengths for main effects and surface offsets by using different (smaller) values than 1 for encoding (player, surface) combos.  By using 0.1 instead of 1, for example, the regression that behaves the same way as the unweighted one would require a coefficient 10x as large, meaning the regularization penalty is also 10x as large.  In this way, we can achieve a 10x increase in the strength of the regularization parameter without modifying the sklearn internals.

In [1]:
from tennis_new.fetch.tennis_explorer.combiner import read_joined

jd = read_joined()

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv_377/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


#### Run Set ELO

Run SetELO first so that we have easy access to training set and validation set and all that

In [2]:
from tennis_new.model.config.elo.global_set_elo import SetELO

set_elo = SetELO()
set_elo.run(jd)

In [3]:
set_elo.validation_evaluation

{'DummyFilter_prediction_AUCMetric': 0.8187031847302881,
 'DummyFilter_prediction_AccuracyMetric': 0.7358520800135314,
 'DummyFilter_prediction_LogLikelihoodMetric': -0.5226366377611569,
 'HasOddsFilter_prediction_AUCMetric': 0.7839029874196454,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7056423354253945,
 'HasOddsFilter_prediction_LogLikelihoodMetric': -0.5594758958654537,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_LogLikelihoodMetric': None,
 'HasOddsFilter_odds_implied_probability_AUCMetric': 0.7937506478103871,
 'HasOddsFilter_odds_implied_probability_AccuracyMetric': 0.7114980299325661,
 'HasOddsFilter_odds_implied_probability_LogLikelihoodMetric': -0.5501844612492598}

#### Define Logit Training X, y

Now we'll need to create a sparse dataset for the logistic regression.  We'll start by making sure we have the right date filtering.  Recall that for ELO models, our training data is the full date range.  We'll have to manually cut the dates for our logit model.

In [4]:
elo_training_set = set_elo.training_filter.filter_data(jd)
elo_validation_set = set_elo.validation_filter.filter_data(set_elo.all_jd)
elo_test_set = set_elo.test_filter.filter_data(set_elo.all_jd)
logit_training_set = elo_training_set[
    elo_training_set['date'] < elo_validation_set['date'].min()
].copy()
(
    (logit_training_set['date'].min(), logit_training_set['date'].max()),
    (elo_validation_set['date'].min(), elo_validation_set['date'].max()),
    (elo_test_set['date'].min(), elo_test_set['date'].max())
)

(('1997-01-01', '2010-12-31'),
 ('2011-01-01', '2014-12-31'),
 ('2015-01-01', '2020-12-21'))

Next, we'll create enumeration of our player ids so that we can easily create a sparse DF

In [5]:
import pandas as pd

all_players = pd.concat([
    logit_training_set[['p1_link', 'date']].rename(columns={'p1_link': 'pid'}).drop_duplicates('pid', keep='first'),
    logit_training_set[['p2_link', 'date']].rename(columns={'p2_link': 'pid'}).drop_duplicates('pid', keep='first')
]).sort_values('date', ascending=True)['pid'].drop_duplicates(keep='first')
PLAYER_MAP = dict(enumerate(all_players))
INV_PLAYER_MAP = {v: k for k, v in PLAYER_MAP.items()}

In [6]:
MISSING_KEY = 'zMISSING'

SURFACE_MAP = dict(enumerate(logit_training_set['surface'].fillna(MISSING_KEY).drop_duplicates().sort_values()))
INV_SURFACE_MAP = {v: k for k, v in SURFACE_MAP.items()}
SURFACE_MAP

{0: 'Various surfaces',
 1: 'clay',
 2: 'grass',
 3: 'hard',
 4: 'indoors',
 5: 'zMISSING'}

#### Now Create Sparse Logit Input

This has previously proved problematic, especially when the surface information is involved.  Below, we 

In [7]:
import numpy as np
from scipy.sparse import csc_matrix

test_player_map = {
    0: '0',
    1: '1',
    2: '2',
    3: '3'
} 
test_surface_map = {
    0: 'grass',
    1: 'hard',
    2: MISSING_KEY
}
test_df = pd.DataFrame({
    'p1_link': ['0', '1', '2', '3'],
    'p2_link': ['1', '2', '0', '2'],
    'p1_sets_won': [2, 1, 3, 2],
    'p2_sets_won': [1, 0, 2, 0],
    'surface': ['grass', None, 'hard', 'hard']
})


MISSING_KEY = 'zMISSING'

class DataProcessor(object):

    def __init__(self, player_map, surface_map=None, surface_multiplier=1.):
        self.player_map = player_map
        self.inv_player_map = {v: k for k, v in self.player_map.items()}
        self.surface_map = surface_map
        if surface_map is not None:
            self.inv_surface_map = {v: k for k, v in self.surface_map.items()}
            self.handle_surfaces = True 
        else:
            self.inv_surface_map = None
            self.handle_surfaces = False
        # Value to use for surface_idx variables.  This is a hack to allow for more regularization of surface offsets
        # than of the main effect
        self.surface_multiplier = surface_multiplier  
            
    def preprocess_data(self, df, use_sets=True):
        _df = df[[
            'p1_link',
            'p2_link',
            'p1_sets_won',
            'p2_sets_won',
            'surface'
        ]].copy()
        _df['__p1_idx__'] = _df['p1_link'].map(self.inv_player_map).astype(int)
        _df['__p2_idx__'] = _df['p2_link'].map(self.inv_player_map).astype(int)
        if use_sets: 
            _df['__clipped_p1_sets__'] = np.clip(_df['p1_sets_won'], 1, None).astype(int)
            _df['__clipped_p2_sets__'] = _df['p2_sets_won'].astype(int)
        else:
            _df['__clipped_p1_sets__'] = 1 
            _df['__clipped_p2_sets__'] = 0 
        _df['__total_sets__'] = _df['__clipped_p1_sets__'] + _df['__clipped_p2_sets__']
        _df['__column_indices__'] = (
            _df['__p1_idx__'].map(lambda x: [x]) +
            _df['__p2_idx__'].map(lambda x: [x])
        ) 
        if self.handle_surfaces:
            _df['__surface_id__'] = _df['surface'].fillna(MISSING_KEY).map(self.inv_surface_map).astype(int)
            _df['__surface_start_idx__'] = (_df['__surface_id__'] + 1) * len(self.player_map)
            _df['__p1_surface_idx__'] = _df['__surface_start_idx__'] + _df['__p1_idx__']
            _df['__p2_surface_idx__'] = _df['__surface_start_idx__'] + _df['__p2_idx__']
            _df['__column_indices__'] += (
                _df['__p1_surface_idx__'].map(lambda x: [x]) +
                _df['__p2_surface_idx__'].map(lambda x: [x])
            )
        return _df
            
    def create_X_y(self, df, use_sets=True):
        preprocessed_df = self.preprocess_data(df, use_sets=use_sets)
        # Stack data
        total_sets = preprocessed_df['__total_sets__'].sum()
        col_indices = np.concatenate(
            np.repeat(
                preprocessed_df['__column_indices__'],
                preprocessed_df['__total_sets__']
            ).map(np.array).values
        )
        if self.handle_surfaces:
            vals_per_row = [1, -1, self.surface_multiplier, -self.surface_multiplier]
            n_cols = len(self.player_map) * (len(self.surface_map) + 1)
        else:
            vals_per_row = [1, -1]
            n_cols = len(self.player_map)

        x_values = np.tile(vals_per_row, total_sets)
        row_indices = np.repeat(range(total_sets), len(vals_per_row))
        assert len(x_values) == len(row_indices) == len(col_indices)

        X = csc_matrix(
            (x_values, (row_indices, col_indices)),
            shape=(total_sets, n_cols)
        )
        
        # create y
        ones = preprocessed_df['__clipped_p1_sets__'].map(lambda x: np.ones(x).astype(int).tolist())
        zeros = preprocessed_df['__clipped_p2_sets__'].map(lambda x: np.zeros(x).astype(int).tolist())
        y = np.concatenate((ones + zeros).tolist())
        return X, y

In [8]:
test_df

,p1_link,p2_link,p1_sets_won,p2_sets_won,surface
0,0,1,2,1,grass
1,1,2,1,0,None
2,2,0,3,2,hard
3,3,2,2,0,hard


In [9]:
test_dp = DataProcessor(test_player_map, surface_map=None)

In [10]:
X, y = test_dp.create_X_y(test_df, use_sets=True)
X.todense(), y

(matrix([[ 1, -1,  0,  0],
         [ 1, -1,  0,  0],
         [ 1, -1,  0,  0],
         [ 0,  1, -1,  0],
         [-1,  0,  1,  0],
         [-1,  0,  1,  0],
         [-1,  0,  1,  0],
         [-1,  0,  1,  0],
         [-1,  0,  1,  0],
         [ 0,  0, -1,  1],
         [ 0,  0, -1,  1]], dtype=int64),
 array([1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1]))

In [11]:
surface_test_dp = DataProcessor(test_player_map, surface_map=test_surface_map, surface_multiplier=10.)
X, y = surface_test_dp.create_X_y(test_df, use_sets=True)
X.todense(), y

(matrix([[  1.,  -1.,   0.,   0.,  10., -10.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  1.,  -1.,   0.,   0.,  10., -10.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  1.,  -1.,   0.,   0.,  10., -10.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,   1.,  -1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
            0.,   0.,   0.,  10., -10.,   0.],
         [ -1.,   0.,   1.,   0.,   0.,   0.,   0.,   0., -10.,   0.,
           10.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,   0.,   1.,   0.,   0.,   0.,   0.,   0., -10.,   0.,
           10.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,   0.,   1.,   0.,   0.,   0.,   0.,   0., -10.,   0.,
           10.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,   0.,   1.,   0.,   0.,   0.,   0.,   0., -10.,   0.,
           10.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,   0.,   1.,   0.,   0.,   0.,   0.,   0., -10., 

Cool, so our the ressults are as expected for our test DF.  Let's try fitting real logistic regressions now (both with and without surface features) on our real data.

In [12]:
dp = DataProcessor(PLAYER_MAP, surface_map=None)
X, y = dp.create_X_y(logit_training_set, use_sets=True)
surface_dp = DataProcessor(PLAYER_MAP, surface_map=SURFACE_MAP, surface_multiplier=0.1)
sX, sy = surface_dp.create_X_y(logit_training_set, use_sets=True)

In [13]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(fit_intercept=False, solver='saga')
logit.fit(
    X, y 
)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
from sklearn.linear_model import LogisticRegression

surface_logit = LogisticRegression(fit_intercept=False, solver='saga')
surface_logit.fit(
    sX, sy 
)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
logit_validation_set = elo_validation_set[
    elo_validation_set['p1_link'].isin(PLAYER_MAP.values()) &
    elo_validation_set['p2_link'].isin(PLAYER_MAP.values()) &
    (elo_validation_set['date'] < '2012-01-01')
].copy()
val_X, val_y = dp.create_X_y(logit_validation_set, use_sets=False)
val_sX, val_sy = surface_dp.create_X_y(logit_validation_set, use_sets=False)

In [16]:
# How does our ELO model perform on the validation set?
(
    (logit_validation_set['prediction'] > 0.5).mean(),
    (logit.predict_proba(val_X)[:, 1] > 0.5).mean(),
    (surface_logit.predict_proba(val_sX)[:, 1] > 0.5).mean()
)

(0.7403709304394236, 0.7056573563422879, 0.7067692581391212)

In [17]:
has_surface_validation_set = logit_validation_set[
    logit_validation_set['surface'].notnull()
].copy()
hs_val_X, hs_val_y = dp.create_X_y(has_surface_validation_set, use_sets=False)
hs_val_sX, hs_val_sy = surface_dp.create_X_y(has_surface_validation_set, use_sets=False)

In [18]:
(
    (has_surface_validation_set['prediction'] > 0.5).mean(),
    (logit.predict_proba(hs_val_X)[:, 1] > 0.5).mean(),
    (surface_logit.predict_proba(hs_val_sX)[:, 1] > 0.5).mean()
)

(0.7098164524039533, 0.6701345024894108, 0.6737014193356617)

Above, we gain a reasonable amount of accuracy from including surface information with a 10x relative regularization penalty.  Let's see if this is a significant difference: 

In [20]:
from scipy.stats import fisher_exact

fisher_exact(
    [
        [
            (logit.predict_proba(hs_val_X)[:, 1] > 0.5).sum(),
            (logit.predict_proba(hs_val_X)[:, 1] <= 0.5).sum(),
        ],
        [
            (surface_logit.predict_proba(hs_val_sX)[:, 1] > 0.5).sum(),
            (surface_logit.predict_proba(hs_val_sX)[:, 1] <= 0.5).sum()
        ]
    ]
)[1]

0.5417424696706867

Above, it looks like this is *not* a significant difference for our size of data.

Coefficient inspection -- Let's take a look at the coefficients and see if the rankings our model recovers are consistent with our expectations.

In [22]:
coef_df = pd.DataFrame({
    'player_idx': range(len(PLAYER_MAP)),
    'coef': logit.coef_[0, :]
})
coef_df['player_link'] = coef_df['player_idx'].map(PLAYER_MAP)
coef_df.sort_values('coef', ascending=False).head()

,player_idx,coef,player_link
1552,1552,5.180929,/player/nadal/
836,836,5.178273,/player/federer/
233,233,4.873466,/player/agassi/
1214,1214,4.856656,/player/roddick/
139,139,4.833060,/player/sampras/


In [23]:
_coefs = surface_logit.coef_[0, :].reshape(
    (len(SURFACE_MAP) + 1, len(PLAYER_MAP)),
).T
s_coef_df = pd.DataFrame(_coefs, columns=['main'] + sorted(SURFACE_MAP.values()))
s_coef_df['player_idx'] = range(len(s_coef_df))
s_coef_df['player_link'] = s_coef_df['player_idx'].map(PLAYER_MAP)
s_coef_df.sort_values('main', ascending=False).head(5)

,main,Various surfaces,clay,grass,hard,indoors,zMISSING,player_idx,player_link
836,5.168220,0.097555,-1.249422,1.257279,1.364180,-0.952770,0.000000,836,/player/federer/
1552,5.139812,0.463256,2.914234,0.074189,-0.320102,-2.222621,-0.394975,1552,/player/nadal/
1214,4.826429,-0.160260,-0.776010,0.584205,0.972544,-0.137837,0.000000,1214,/player/roddick/
233,4.819809,0.088843,-0.712436,-0.002006,1.183215,-0.075635,0.000000,233,/player/agassi/
139,4.790050,-0.144149,-2.086960,1.057952,0.803580,0.848583,0.000000,139,/player/sampras/


Let's see who has the biggest surface-specific adjustments:

In [24]:
s_coef_df.sort_values('clay', ascending=False)[['main', 'clay', 'player_link']].head(10)

,main,clay,player_link
8501,3.286023,3.528068,/player/bellucci/
1497,3.605316,3.486916,/player/starace/
1005,3.517185,3.179439,/player/volandri/
326,3.477992,3.017018,/player/portas/
1552,5.139812,2.914234,/player/nadal/
982,3.279763,2.635250,/player/vassallo-arguello/
86,3.888661,2.622425,/player/mantilla/
151,3.337208,2.540238,/player/blanco/
1168,3.726788,2.519917,/player/acasuso/
4779,3.919391,2.513787,/player/almagro/


In [25]:
surface_quality_df = s_coef_df.copy()
for col in SURFACE_MAP.values():
    surface_quality_df[col] = surface_quality_df['main'] + surface_quality_df[col]

surface_quality_df.sort_values('clay', ascending=False).head(5)

,main,Various surfaces,clay,grass,hard,indoors,zMISSING,player_idx,player_link
1552,5.139812,5.603068,8.054046,5.214000,4.819710,2.917190,4.744837,1552,/player/nadal/
1497,3.605316,4.338144,7.092232,2.873911,2.073353,2.081617,3.533171,1497,/player/starace/
8501,3.286023,3.044252,6.814091,3.348598,3.866585,3.491467,-0.520251,8501,/player/bellucci/
1005,3.517185,3.751706,6.696624,3.143398,1.865084,2.480833,3.517185,1005,/player/volandri/
86,3.888661,3.810278,6.511087,3.023639,3.462724,3.073589,3.839517,86,/player/mantilla/


In [26]:
surface_quality_df.sort_values('hard', ascending=False).head(5)

,main,Various surfaces,clay,grass,hard,indoors,zMISSING,player_idx,player_link
836,5.168220,5.265775,3.918798,6.425499,6.532399,4.215450,5.168220,836,/player/federer/
3325,4.507004,4.702344,2.952930,4.721055,6.518082,5.532682,3.065629,3325,/player/murray/
6873,4.700031,4.715040,5.238796,4.598778,6.302329,4.457974,3.357273,6873,/player/djokovic/
8879,4.172113,4.476857,4.628081,4.192386,6.156748,4.214228,1.781587,8879,/player/del-potro/
222,4.019767,3.394987,2.571767,4.448368,6.018731,4.066957,4.019767,222,/player/kiefer/


In [27]:
surface_quality_df.sort_values('grass', ascending=False).head(5)

,main,Various surfaces,clay,grass,hard,indoors,zMISSING,player_idx,player_link
836,5.168220,5.265775,3.918798,6.425499,6.532399,4.215450,5.168220,836,/player/federer/
139,4.790050,4.645901,2.703089,5.848001,5.593629,5.638633,4.790050,139,/player/sampras/
152,4.657749,4.993575,3.479018,5.689429,5.159737,4.432763,4.657749,152,/player/hewitt/
1214,4.826429,4.666169,4.050419,5.410634,5.798973,4.688592,4.826429,1214,/player/roddick/
4,4.466138,4.681318,3.764912,5.380417,4.855823,4.094836,4.466138,4,/player/rafter/


In [28]:
surface_quality_df.sort_values('indoors', ascending=False).head(5)

,main,Various surfaces,clay,grass,hard,indoors,zMISSING,player_idx,player_link
88,4.122040,3.567408,1.513612,5.220850,4.658481,6.062053,4.122040,88,/player/rusedski/
77,3.597507,3.785257,2.505569,3.270266,2.827144,5.959048,3.597507,77,/player/rosset/
351,4.209054,3.769686,2.206456,4.812374,4.970697,5.706960,4.209054,351,/player/kafelnikov/
139,4.790050,4.645901,2.703089,5.848001,5.593629,5.638633,4.790050,139,/player/sampras/
234,4.393801,4.288143,3.683437,4.260212,4.632614,5.543977,4.393801,234,/player/krajicek/


Now, we can see that we only have a modest accuracy improvement from including the surface information.  However, the surface-by-surface rankings produced by adding this information largely seem to make sense (Nadal is king of clay, Sampras appears very high on grass, etc.)  The gains will likely be larger when we have surface information for every match.